In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# load packages
import sys # access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import numpy as np # scientific computing
print("NumPy version: {}". format(np.__version__))

import pandas as pd # data processing and analysis 
print("pandas version: {}". format(pd.__version__))

import matplotlib
import matplotlib.pyplot as plt # plotting
%matplotlib inline 
print("matplotlib version: {}". format(matplotlib.__version__))
 
import sklearn # machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

from itertools import product
from xgboost import XGBRegressor
from xgboost import plot_importance
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

print("----------")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Python version: 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
NumPy version: 1.18.5
pandas version: 1.1.2
matplotlib version: 3.2.1
scikit-learn version: 0.23.2
----------
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv
/kaggle/input/pfs-model/finalized_model.sav
/kaggle/input/pfs-model/__results__.html
/kaggle/input/pfs-model/submission.csv
/kaggle/input/pfs-model/predictionsLGB.csv
/kaggle/input/pfs-model/__notebook__.ipynb
/kaggle/input/pfs-model/custom.css
/kaggle/input/pfs-model/__output__.json
/kaggle/input/pfs-model/__results___files/__results___16_0.png

# Load Data

In [2]:
# read csv file into data frames
#df_items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
#df_shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv') # read cleaned version instead later
#df_sales = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
df_test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
#df_item_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
#df_sample_submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')

# my versions
all_data = pd.read_pickle('../input/pfs-featureengineering1/all_data_withFeatures.pkl')
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc_kf,item_target_enc_loo,item_target_enc_sm,item_target_enc_ems,item_category_id,days_per_month,...,monthly_transactions_lag_4,monthly_transactions_lag_5,monthly_transactions_lag_12,price_trend,acc_price_lag,name_sqbrac,name_brac,word1,word12,type
0,59,22154,0,1.0,0.084746,0.108209,0.145052,0.333984,37,31,...,0.0,0.0,0.0,0.0,0.0,4,548,4293,8301,0
1,59,2552,0,0.0,0.021583,0.018248,0.102396,0.333984,58,31,...,0.0,0.0,0.0,0.0,0.0,4,41,292,530,0
2,59,2554,0,0.0,0.333984,0.022727,0.237230,0.333984,58,31,...,0.0,0.0,0.0,0.0,0.0,4,41,292,530,0
3,59,2555,0,0.0,0.056995,0.066508,0.094802,0.333984,56,31,...,0.0,0.0,0.0,0.0,0.0,4,1511,292,530,0
4,59,2564,0,0.0,0.040307,0.058599,0.089614,0.333984,59,31,...,0.0,0.0,0.0,0.0,0.0,4,1124,292,530,0


# First Level Models
Lightgbm data comes from PFS-model, my main modelling notebook. Here a Linear Regression is added as second first level model.

In [3]:
all_data.columns

Index(['shop_id', 'item_id', 'date_block_num', 'target', 'item_target_enc_kf',
       'item_target_enc_loo', 'item_target_enc_sm', 'item_target_enc_ems',
       'item_category_id', 'days_per_month', 'month', 'city_code',
       'item_bcat_code', 'item_scat_code', 'target_lag_1', 'target_lag_2',
       'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_12',
       'month_item_sum_lag_1', 'month_item_sum_lag_2', 'month_item_sum_lag_3',
       'month_item_sum_lag_4', 'month_item_sum_lag_5', 'month_item_sum_lag_12',
       'month_shop_sum_lag_1', 'month_shop_sum_lag_2', 'month_shop_sum_lag_3',
       'month_shop_sum_lag_4', 'month_shop_sum_lag_5', 'month_shop_sum_lag_12',
       'month_sum_lag_1', 'month_sum_lag_2', 'month_sum_lag_3',
       'month_sum_lag_4', 'month_sum_lag_5', 'month_sum_lag_12',
       'monthly_transactions_lag_1', 'monthly_transactions_lag_2',
       'monthly_transactions_lag_3', 'monthly_transactions_lag_4',
       'monthly_transactions_lag_5', 'monthly_trans

In [4]:
#select columns for training data
columns = ['shop_id', 'item_id', 'date_block_num', #'item_target_enc_kf',
       #'item_target_enc_loo', 'item_target_enc_sm', 'item_target_enc_ems',
       'item_category_id', 'days_per_month', 'month', 'city_code',
       'item_bcat_code', 'item_scat_code', 'target_lag_1', 'target_lag_2',
       'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_12',
       'month_item_sum_lag_1', 'month_item_sum_lag_2', 'month_item_sum_lag_3',
       'month_item_sum_lag_4', 'month_item_sum_lag_5', 'month_item_sum_lag_12',
       'month_shop_sum_lag_1', 'month_shop_sum_lag_2', 'month_shop_sum_lag_3',
       'month_shop_sum_lag_4', 'month_shop_sum_lag_5', 'month_shop_sum_lag_12',
       'month_sum_lag_1', 'month_sum_lag_2', 'month_sum_lag_3',
       'month_sum_lag_4', 'month_sum_lag_5', 'month_sum_lag_12',
       'monthly_transactions_lag_1', 'monthly_transactions_lag_2',
       'monthly_transactions_lag_3', 'monthly_transactions_lag_4',
       'monthly_transactions_lag_5', 'monthly_transactions_lag_12',
       'price_trend', 'acc_price_lag', 'name_sqbrac', 'name_brac', #'word1',
       #'word12', 'type'
          ]
if 'target' in columns:
    print("You are a stuipd moron!")

In [5]:
# lets use month 33 as validation data and month 3/12-32 as training data, drop frist months due to lag feature

X_val = all_data[all_data.date_block_num == 33] # month 33 data
y_val = X_val["target"]
X_val = X_val[columns]

#X_train = all_data[(all_data.date_block_num >2) & (all_data.date_block_num < 33)] # month 3-32 data
X_train = all_data[(all_data.date_block_num >11) & (all_data.date_block_num < 33)] # month 12-32 data
y_train = X_train["target"]
X_train = X_train[columns]

X_test = all_data[all_data.date_block_num == 34][columns]

In [6]:
X_train.head()

,shop_id,item_id,date_block_num,item_category_id,days_per_month,month,city_code,item_bcat_code,item_scat_code,target_lag_1,...,monthly_transactions_lag_1,monthly_transactions_lag_2,monthly_transactions_lag_3,monthly_transactions_lag_4,monthly_transactions_lag_5,monthly_transactions_lag_12,price_trend,acc_price_lag,name_sqbrac,name_brac
4488710,54,10297,12,37,31,1,25,5,1,3.0,...,143245.0,96736.0,0.0,0.0,0.0,0.0,-0.010032,0.037453,4,548
4488711,54,10296,12,38,31,1,25,5,1,0.0,...,143245.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.084995,4,337
4488712,54,10298,12,40,31,1,25,5,4,21.0,...,143245.0,96736.0,94202.0,0.0,0.0,0.0,0.006816,0.438594,4,41
4488713,54,10300,12,37,31,1,25,5,1,1.0,...,143245.0,96736.0,94202.0,0.0,0.0,0.0,-0.006491,0.160869,4,548
4488714,54,10284,12,57,31,1,25,7,8,0.0,...,143245.0,96736.0,94202.0,96137.0,104772.0,0.0,-0.124060,-0.073800,4,763


In [7]:
ts = time.time()

lr = LinearRegression()
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_val)
print('Test R-squared for linreg is %f' % r2_score(y_val, pred_lr))
print('Test RMSE for linreg is %f' % mean_squared_error(y_val, pred_lr,squared=False))

time.time() - ts

Test R-squared for linreg is 0.150965
Test RMSE for linreg is 1.046888


8.240033864974976

In [8]:
print(pred_lr.max())
print(pred_lr.min())
print('Coefficients: \n', lr.coef_)
print('Intercept: \n', lr.intercept_)

39.931746517029744
-10.64046232904296
Coefficients: 
 [-1.04145219e-03 -3.50770611e-06 -2.61877469e-03  7.25949022e-03
  5.04764359e-04  1.39737457e-02  3.12124747e-03 -1.02352354e-01
 -9.08891275e-04  5.57969979e-02  1.86081362e-02  9.36381070e-03
  2.98650553e-03 -2.14334304e-03 -1.63623186e-02  2.27514913e-03
  7.20784092e-05  1.95003474e-04  6.22529643e-05 -5.81822762e-05
  2.65767754e-04  1.04875135e-04  2.09077719e-05  2.81181602e-06
 -7.62203508e-06 -7.32896979e-06 -2.16077499e-06 -5.46762630e-07
 -1.53610921e-07  5.46343875e-06 -1.21568517e-06  3.53137681e-06
 -1.10225862e-06 -8.16825011e-07 -7.84732117e-07 -7.28399323e-06
  1.27488274e-06 -4.67615604e-06  8.86498589e-07  4.25591357e-05
  1.00709442e-01  4.68303509e-04 -9.14861817e-05]
Intercept: 
 0.5453412402890511


In [9]:
# run linear regression again with X_train + X_val, i.e. data from month 12-33
ts = time.time()
X_train_n = all_data[(all_data.date_block_num >11) & (all_data.date_block_num < 34)]
y_train_n = X_train_n["target"]
X_train_n = X_train_n[columns]

lr = LinearRegression()
lr.fit(X_train_n, y_train_n)
pred1stlevel_lr = lr.predict(X_test)

time.time() - ts

9.45840859413147

In [10]:
pred1stlevel_lgb = pd.read_csv('../input/pfs-model/predictionsLGB.csv')
pred1stlevel_lgb.drop(columns='ID', inplace=True)

In [11]:
# contains the predictions from linear regression and xgboost for month 34, i.e. the test set
X_test_level2 = np.c_[pred1stlevel_lr, pred1stlevel_lgb] 

In [12]:
X_test_level2

array([[0.42273521, 0.49835429],
       [0.28651476, 0.35191516],
       [0.60307733, 1.36824878],
       ...,
       [0.09119959, 0.03589282],
       [0.08322124, 0.03058733],
       [0.08319306, 0.0212149 ]])

# Train Meta Features

In [13]:
X_train.tail()

,shop_id,item_id,date_block_num,item_category_id,days_per_month,month,city_code,item_bcat_code,item_scat_code,target_lag_1,...,monthly_transactions_lag_1,monthly_transactions_lag_2,monthly_transactions_lag_3,monthly_transactions_lag_4,monthly_transactions_lag_5,monthly_transactions_lag_12,price_trend,acc_price_lag,name_sqbrac,name_brac
10675627,27,21279,32,61,30,9,12,8,20,0.0,...,0.0,0.0,54617.0,54548.0,0.0,0.0,0.0,0.000000,4,41
10675628,27,21283,32,61,30,9,12,8,20,0.0,...,0.0,0.0,54617.0,0.0,56274.0,73157.0,0.0,0.007758,4,41
10675629,27,21352,32,37,30,9,12,5,1,0.0,...,0.0,55549.0,0.0,54548.0,0.0,73157.0,0.0,-0.532101,4,548
10675630,27,21284,32,61,30,9,12,8,20,0.0,...,0.0,0.0,54617.0,0.0,0.0,0.0,0.0,0.000000,4,41
10675631,27,21105,32,61,30,9,12,8,20,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4,41


In [14]:
# target for meta features, i.e. the target variable for date_block_num 27-33
y_train_level2 = all_data.target[all_data.date_block_num.isin([27, 28, 29, 30, 31, 32, 33])]
y_train_level2.shape

(1614364,)

In [15]:
# define 2nd model, same parameters as for first level prediction 
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':1 
              }

Run either next cell if the 2nd level feature matrix needs to be re-generated or use next cell to load the existing one. Generating the 2nd level feature matrix tages very long and is not needed every time.

Change the next 2 cells back to code to generate the 2nd level feature matrix and save it.



ts = time.time()
### create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 2])

pred_lr_acc = []
pred_lgb_acc = []

### Now fill `X_train_level2` with metafeatures
for cur_block_num in [27, 28, 29, 30, 31, 32, 33]:
    
    print(cur_block_num)
    
    '''
        1. Split `X_train` into parts 
        2. Fit linear regression 
        3. Fit Lightgbm and put predictions          
        4. Store predictions from 2. and 3. in a list         
    '''      
    
    # 1.
    X_train_split = all_data.loc[(all_data.date_block_num > 11) & (all_data.date_block_num < cur_block_num)] # training data: date_block_num 12-26/27/....32
    X_test_split = all_data.loc[all_data.date_block_num == cur_block_num]  # testing data: date_block_num = 27/28/....33
    y_train_split = all_data.loc[(all_data.date_block_num > 11) & (all_data.date_block_num < cur_block_num), 'target'].values #----- month numbers not yet consistant
    y_test_split =  all_data.loc[all_data.date_block_num == cur_block_num, 'target'].values
    X_train_split = X_train_split[columns] # get rid of 'target'column
    X_test_split = X_test_split[columns] # get rid of 'target' column
    print(X_train_split.shape)   
    print(y_train_split.shape)
    print(X_test_split.shape)
    print(y_test_split.shape)
    
    #2.
    lr.fit(X_train_split.values, y_train_split)
    pred_lr = lr.predict(X_test_split.values)
    # the predictions are made for exactly one month
    print(pred_lr.shape)
    pred_lr.tolist()
    
    #3.
    lgbmodel = lgb.train(lgb_params, lgb.Dataset(X_train_split, label=y_train_split), 100)
    pred_lgb = lgbmodel.predict(X_test_split)
    # the predictions are made for exactly one month
    print(pred_lgb.shape)
    pred_lgb.tolist()
    
    #4.
    pred_lr_acc.extend(pred_lr) # add monthly predictions to a list
    pred_lgb_acc.extend(pred_lgb)
    #print(pred_lr)
    #print(pred_lgb)

X_train_level2 = np.c_[pred_lr_acc, pred_lgb_acc]

time.time() - ts

### save X_train_level2 
X_train_level2df = pd.DataFrame(X_train_level2)
X_train_level2df.columns = ['pred_lr', 'pred_lgb']
date_block_num_lev2 = all_data.date_block_num[all_data.date_block_num.isin([27, 28, 29, 30, 31, 32, 33])]
X_train_level2df['date_block_num'] = date_block_num_lev2.values
X_train_level2df.to_csv('X_train_level2.csv', index=False)

In [16]:
# make scatter plot, take long
#x = pred_lr_acc  # =X_train_level2[:,0]
#y = pred_lgb_acc # =X_train_level2[:,1]

#plt.scatter(x, y)
#plt.show()

In [17]:
# read in X_train_level2df from disk as it takes ages to create it
X_train_level2df = pd.read_csv('../input/x-train-level2/X_train_level2.csv')
X_train_level2df.head()
X_train_level2 = X_train_level2df.drop(columns="date_block_num")
X_train_level2 = X_train_level2.to_numpy() # convert back to numpy array
X_train_level2

array([[0.26024273, 0.58960767],
       [1.03049882, 2.3276679 ],
       [0.38913534, 0.44281764],
       ...,
       [0.18638149, 0.12290865],
       [0.18685823, 0.0554179 ],
       [0.18248646, 0.05290426]])

In [18]:
# get y_train_level2
y_train_level2 = all_data.loc[(all_data.date_block_num.isin([27, 28, 29, 30, 31, 32, 33])),'target']
y_train_level2.shape

(1614364,)

# Ensembling
## Simple Convex Mix
I tried to different metrics, R2 and rsme. They gave me opposite extreme results for alpha. Still dont understand why. Suggestions are welcome!

In [19]:
### using metrix r2
ts = time.time()
alphas_to_try = np.linspace(0, 1, 1001)
alpha_r2_score = []

for alpha in alphas_to_try:
    mixed_pred = alpha * X_train_level2[:,0] + (1-alpha) * X_train_level2[:,1]
    r2_score_a = r2_score(y_train_level2, mixed_pred)
    alpha_r2_score.extend([[alpha, r2_score_a]])

alpha_r2_score = pd.DataFrame(alpha_r2_score) # maybe there is a faster way using arrays or lists, but I don't know it

r2_train_simple_mix = alpha_r2_score[1].max() # get maximum r2 value
best_alpha = alpha_r2_score[0][alpha_r2_score[1] == r2_train_simple_mix].values # go to row with maximum r2 value and get alpha for it

print('Best alpha: %f; Corresponding r2 score on train: %f' % (best_alpha, r2_train_simple_mix))
time.time() - ts

Best alpha: 0.000000; Corresponding r2 score on train: 0.396893


23.37584376335144

### using metric rmse
ts = time.time()
alphas_to_try = np.linspace(0, 1, 1001)
alpha_rmse_score = []

for alpha in alphas_to_try:
    mixed_pred = alpha * X_train_level2[:,0] + (1-alpha) * X_train_level2[:,1]
    rmse_score_a = mean_squared_error(y_train_level2, mixed_pred,squared=False)
    alpha_rmse_score.extend([[alpha, rmse_score_a]])

alpha_rmse_score = pd.DataFrame(alpha_rmse_score) # maybe there is a faster way using arrays or lists, but I don't know it

rsme_train_simple_mix = alpha_rmse_score[1].max() # get maximum rmse value
best_alpha = alpha_rmse_score[0][alpha_rmse_score[1] == rsme_train_simple_mix].values # go to row with maximum r2 value and get alpha for it

print('Best alpha: %f; Corresponding rmse score on train: %f' % (best_alpha, rsme_train_simple_mix))
time.time() - ts

Metric r2: The linreg is so bad that the results are not used, haha. 

Metric rmse: Now only the linreg is used. pfff, why?

In [20]:
#calculate predictions for simple mix
test_preds = best_alpha * X_test_level2[:,0] + (1-best_alpha) * X_test_level2[:,1]

In [21]:
# save predictions from simple mix 
preds_ensemb_simple =  pd.DataFrame({
    "ID": df_test.index, 
    "item_cnt_month": test_preds
})
preds_ensemb_simple.to_csv('preds_ensemb_simple.csv', index=False)

## Stacking

In [22]:
# my comments: run Linear Regression on X_train_level2 (2 features)
# training data is from date_block_num 27-33, test data is date_block_num 34
lr = LinearRegression()
lr.fit(X_train_level2, y_train_level2)
y_hat_level2 = lr.predict(X_test_level2)

In [23]:
y_hat_level2.shape

(214200,)

In [24]:
# create final submission file
submission = pd.DataFrame({
    "ID": df_test.index, 
    "item_cnt_month": y_hat_level2
})
submission.to_csv('stacked_submission.csv', index=False)

In [25]:
import pickle
filename = 'finalized_ensemble_model.sav'
pickle.dump(lr, open(filename, 'wb'))

In [26]:
train_preds = lr.predict(X_train_level2)
r2_train_stacking = r2_score(y_train_level2, train_preds)

print('Train R-squared for stacking is %f' % r2_train_stacking)
print('Train RMSE for linreg is %f' % mean_squared_error(y_train_level2, train_preds,squared=False))


Train R-squared for stacking is 0.397220
Train RMSE for linreg is 0.842216
